# Password Cracking Tool for Mock2 Files

This notebook attempts to crack password-protected Office documents and PDFs using a wordlist.

## Features
- Filters passwords by length (6-12 characters)
- Tests passwords against 25 password-protected files
- Supports Office files (DOCX, PPTX, XLSX) and PDFs
- Real-time progress tracking
- Saves results to file

## Password Criteria
- **Length**: 6 to 12 characters
- **Characters**: Alphanumeric + Symbols
- **Patterns**: Human-like (names, dates, places)

## 1. Install Dependencies

Run this cell first to install required packages:

In [ ]:
!pip install -q msoffcrypto-tool pikepdf

## 2. Import Libraries

In [ ]:
import os
import sys
from pathlib import Path
from datetime import datetime
from IPython.display import display, HTML, clear_output
import time

try:
    import msoffcrypto
    print("✓ msoffcrypto-tool installed")
except ImportError:
    print("✗ msoffcrypto-tool NOT installed - run cell 1 above")

try:
    import pikepdf
    print("✓ pikepdf installed")
except ImportError:
    print("✗ pikepdf NOT installed - run cell 1 above")

## 3. Define Password Cracker Class

In [ ]:
class PasswordCracker:
    def __init__(self, wordlist_path, target_dir, min_length=6, max_length=12):
        self.wordlist_path = Path(wordlist_path)
        self.target_dir = Path(target_dir)
        self.min_length = min_length
        self.max_length = max_length
        self.results = []
        self.attempts = 0
        self.current_file = None
        self.current_progress = 0

    def load_passwords(self):
        """Load passwords from wordlist file and filter by length"""
        print(f"📂 Loading passwords from: {self.wordlist_path.name}")
        print(f"🔍 Filtering passwords: length {self.min_length}-{self.max_length} characters")
        try:
            with open(self.wordlist_path, 'r', encoding='utf-8', errors='ignore') as f:
                all_passwords = [line.strip() for line in f if line.strip()]
            
            # Filter by length
            passwords = [p for p in all_passwords if self.min_length <= len(p) <= self.max_length]
            
            print(f"✓ Loaded {len(all_passwords):,} total passwords")
            print(f"✓ Filtered to {len(passwords):,} passwords (length {self.min_length}-{self.max_length})")
            print(f"⏩ Skipped {len(all_passwords) - len(passwords):,} passwords outside length range")
            return passwords
        except Exception as e:
            print(f"✗ Error loading wordlist: {e}")
            return []

    def get_target_files(self):
        """Get all files from target directory"""
        print(f"\n📁 Scanning target directory: {self.target_dir.name}")
        files = list(self.target_dir.glob("*"))
        office_files = [f for f in files if f.suffix.lower() in ['.docx', '.pptx', '.xlsx']]
        pdf_files = [f for f in files if f.suffix.lower() == '.pdf']
        print(f"✓ Found {len(office_files)} Office files and {len(pdf_files)} PDF files")
        return office_files, pdf_files

    def try_office_password(self, file_path, password):
        """Try to decrypt an Office file with given password"""
        try:
            with open(file_path, 'rb') as f:
                office_file = msoffcrypto.OfficeFile(f)
                office_file.load_key(password=password)
                # Try to decrypt to verify password
                with open('/dev/null', 'wb') as out:
                    office_file.decrypt(out)
            return True
        except Exception:
            return False

    def try_pdf_password(self, file_path, password):
        """Try to open a PDF file with given password"""
        try:
            with pikepdf.open(file_path, password=password) as pdf:
                return True
        except pikepdf.PasswordError:
            return False
        except Exception:
            return False

    def crack_file(self, file_path, passwords):
        """Attempt to crack a single file"""
        file_name = file_path.name
        file_ext = file_path.suffix.lower()
        self.current_file = file_name

        print(f"\n🔐 Cracking: {file_name}")
        print(f"⏳ Testing {len(passwords):,} passwords...")

        start_time = datetime.now()
        last_update = time.time()

        for idx, password in enumerate(passwords, 1):
            self.attempts += 1
            self.current_progress = idx

            # Progress indicator every 500 attempts or every 2 seconds
            current_time = time.time()
            if idx % 500 == 0 or (current_time - last_update) >= 2:
                progress_pct = (idx / len(passwords)) * 100
                print(f"   Progress: {idx:,}/{len(passwords):,} ({progress_pct:.1f}%) - Testing: {password[:20]}...", end='\r')
                last_update = current_time

            # Try the appropriate method based on file type
            success = False
            if file_ext in ['.docx', '.pptx', '.xlsx']:
                success = self.try_office_password(file_path, password)
            elif file_ext == '.pdf':
                success = self.try_pdf_password(file_path, password)

            if success:
                elapsed = (datetime.now() - start_time).total_seconds()
                result = {
                    'file': file_name,
                    'password': password,
                    'attempts': idx,
                    'time': elapsed
                }
                self.results.append(result)
                print(f"\n✅ SUCCESS! Password found for {file_name}")
                print(f"   🔑 Password: {password}")
                print(f"   🎯 Attempts: {idx:,}")
                print(f"   ⏱️  Time: {elapsed:.2f} seconds")
                return True

        elapsed = (datetime.now() - start_time).total_seconds()
        print(f"\n❌ Failed to crack {file_name} after {len(passwords):,} attempts ({elapsed:.2f}s)")
        return False

    def run(self):
        """Main execution method"""
        print("="*70)
        print("🔓 Password Cracking Tool - Mock2 Files")
        print("="*70)

        # Load passwords
        passwords = self.load_passwords()
        if not passwords:
            print("❌ No passwords loaded!")
            return

        # Get target files
        office_files, pdf_files = self.get_target_files()
        all_files = office_files + pdf_files

        if not all_files:
            print("❌ No files found to crack!")
            return

        print(f"\n🚀 Starting crack attempt on {len(all_files)} files...")
        print(f"📊 Total combinations: {len(passwords):,} passwords × {len(all_files)} files = {len(passwords) * len(all_files):,}")

        start_time = datetime.now()

        # Try to crack each file
        for file_path in all_files:
            self.crack_file(file_path, passwords)

        # Summary
        total_time = (datetime.now() - start_time).total_seconds()
        print("\n" + "="*70)
        print("📋 SUMMARY")
        print("="*70)
        print(f"Total files tested: {len(all_files)}")
        print(f"✅ Successfully cracked: {len(self.results)}")
        print(f"❌ Failed to crack: {len(all_files) - len(self.results)}")
        print(f"🔢 Total attempts: {self.attempts:,}")
        print(f"⏱️  Total time: {total_time:.2f} seconds ({total_time/60:.1f} minutes)")

        if self.results:
            print("\n" + "="*70)
            print("🎉 CRACKED FILES")
            print("="*70)
            for i, result in enumerate(self.results, 1):
                print(f"\n{i}. File: {result['file']}")
                print(f"   🔑 Password: {result['password']}")
                print(f"   🎯 Attempts: {result['attempts']:,}")
                print(f"   ⏱️  Time: {result['time']:.2f}s")

            # Save results to file
            output_file = Path.cwd() / 'cracked_passwords.txt'
            with open(output_file, 'w') as f:
                f.write("Cracked Passwords - Mock2 Files\n")
                f.write("="*70 + "\n")
                f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
                for result in self.results:
                    f.write(f"File: {result['file']}\n")
                    f.write(f"Password: {result['password']}\n")
                    f.write(f"Attempts: {result['attempts']:,}\n")
                    f.write(f"Time: {result['time']:.2f}s\n\n")
            print(f"\n💾 Results saved to: {output_file}")
        else:
            print("\n⚠️  No files were cracked. The passwords may not be in the wordlist.")

        print("="*70)

print("✓ PasswordCracker class defined")

## 4. Configure Paths

In [ ]:
# Set paths
WORDLIST = "/home/tu-mbg/Documents/cc-show-around/PD/Password-Decryption/indian-wordlist/all-indian.txt"
TARGET_DIR = "/home/tu-mbg/Documents/cc-show-around/PD/Password-Decryption/Files/Mock2"

# Verify paths exist
if Path(WORDLIST).exists():
    print(f"✓ Wordlist found: {Path(WORDLIST).name}")
else:
    print(f"✗ Wordlist not found: {WORDLIST}")

if Path(TARGET_DIR).exists():
    file_count = len(list(Path(TARGET_DIR).glob("*")))
    print(f"✓ Target directory found: {Path(TARGET_DIR).name} ({file_count} files)")
else:
    print(f"✗ Target directory not found: {TARGET_DIR}")

## 5. Run Password Cracker

⚠️ **Note**: This may take 5-15 minutes depending on your system.

**Password Requirements:**
- Length: 6 to 12 characters
- Characters: Alphanumeric + Symbols
- Patterns: Human-like (names, dates, places)

The script will:
- Filter passwords to only test length 6-12 characters
- Test filtered passwords against 25 files
- Show progress updates every 500 passwords
- Display results immediately when passwords are found

In [ ]:
# Create and run cracker with password length filter (6-12 characters)
cracker = PasswordCracker(WORDLIST, TARGET_DIR, min_length=6, max_length=12)
cracker.run()

## 6. View Results (Optional)

Display cracked passwords if any were found:

In [ ]:
# Check if results file exists
results_file = Path.cwd() / 'cracked_passwords.txt'

if results_file.exists():
    print("📄 Reading cracked_passwords.txt:\n")
    with open(results_file, 'r') as f:
        print(f.read())
else:
    print("ℹ️  No results file found. Run the cracker first (cell 5).")